In [2]:
import pandas as pd
import duckdb as db

In [3]:
aircraft_csv = r"C:\Users\USER\Desktop\anac_reg_flights\misc_data\sources\aircraft_and_manufacturers.csv"
vra_2017_2023 = r"C:\Users\USER\Desktop\vra_bfd_dataset\VRA_PARQUET\vra_2017_2023.parquet"

In [4]:
aircraft_csv_df = pd.read_csv(aircraft_csv)

In [8]:
vra_df = pd.read_parquet(vra_2017_2023)

In [4]:
aircraft_csv_df.head()

,IATACode,ICAOCode,Model,Aircraft_Manufacturer,WingType,Type,Manufacturer
0,32N,A20N,Airbus A320neo,Airbus,Fixed Wing,Airplane,Airbus
1,A4F,A124,Antonov AN-124 Ruslan,Antonov,Fixed Wing,Airplane,Antonov
2,351,A35K,Airbus A350-1000,Airbus,Fixed Wing,Airplane,Airbus
3,339,A339,Airbus A330-900neo,Airbus,Fixed Wing,Airplane,Airbus
4,BH2,B212,Bell 212,Bell,Rotary Wing,Helicopter,Bell


In [9]:
vra_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6057059 entries, 0 to 83564
Data columns (total 20 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Sigla ICAO Empresa Aérea      object
 1   Empresa Aérea                 object
 2   Número Voo                    object
 3   Código DI                     object
 4   Código Tipo Linha             object
 5   Modelo Equipamento            object
 6   Número de Assentos            object
 7   Sigla ICAO Aeroporto Origem   object
 8   Descrição Aeroporto Origem    object
 9   Partida Prevista              object
 10  Partida Real                  object
 11  Sigla ICAO Aeroporto Destino  object
 12  Descrição Aeroporto Destino   object
 13  Chegada Prevista              object
 14  Chegada Real                  object
 15  Situação Voo                  object
 16  Justificativa                 object
 17  Referência                    object
 18  Situação Partida              object
 19  Situaçã

Extract aircraft used by AZU only

In [14]:
vra_azu_aircrafts = db.sql(
    """
    select
        distinct (coalesce(trim("Modelo Equipamento"),'No Information')) as aircraft_icao,
    from vra_df
    where
        trim("Código DI") = '0' and
        trim("Sigla ICAO Empresa Aérea") = 'AZU'
    order by aircraft_icao asc
    """
).df()

In [67]:
vra_azu_aircrafts

,aircraft_icao
0,A20N
1,A21N
2,A320
3,A321
4,A330
5,A332
6,A339
7,A359
8,AN22
9,AT72


Save as csv for future use

In [17]:
vra_azu_aircrafts.to_csv(r"C:\Users\USER\Desktop\anac_reg_flights\data_dictionary\azu_aircrafts_icao.csv",index=False)

Add additional aircraft information using "aircraft_and_manufacturers.csv"

In [5]:
azu_aircraft_icao = pd.read_csv(r"C:\Users\USER\Desktop\anac_reg_flights\data_dictionary\azu_aircrafts_icao.csv")

In [6]:
azu_aircraft_icao.head()

,aircraft_icao
0,A20N
1,A21N
2,A320
3,A321
4,A330


In [7]:
aircraft_csv_df.head()

,IATACode,ICAOCode,Model,Aircraft_Manufacturer,WingType,Type,Manufacturer
0,32N,A20N,Airbus A320neo,Airbus,Fixed Wing,Airplane,Airbus
1,A4F,A124,Antonov AN-124 Ruslan,Antonov,Fixed Wing,Airplane,Antonov
2,351,A35K,Airbus A350-1000,Airbus,Fixed Wing,Airplane,Airbus
3,339,A339,Airbus A330-900neo,Airbus,Fixed Wing,Airplane,Airbus
4,BH2,B212,Bell 212,Bell,Rotary Wing,Helicopter,Bell


In [8]:
vra_azu_aircrafts_csv = db.sql(
    """
    select
        a.aircraft_icao,
        coalesce(trim(b.IATACode),'Unidentified') as aircraft_iata,
        coalesce(trim(b.Model),'Unidentified') as model_name,
        coalesce(trim(b.Aircraft_Manufacturer),'Unidentified') as manufacturer
    from azu_aircraft_icao as a
    left join aircraft_csv_df as b on a.aircraft_icao = b.ICAOCode
    """
).df()

In [9]:
vra_azu_aircrafts_csv

,aircraft_icao,aircraft_iata,model_name,manufacturer
0,A20N,32N,Airbus A320neo,Airbus
1,A339,339,Airbus A330-900neo,Airbus
2,AT72,AT7,Aerospatiale/Alenia ATR 72,Aerospatiale
3,B734,734,Boeing 737-400,Boeing
4,A321,321,Airbus A321,Airbus
5,AT76,ATR,Aerospatiale/Alenia ATR 72-600,Aerospatiale
6,A320,32A,Airbus A320 (sharklets),Airbus
7,A359,359,Airbus A350-900,Airbus
8,A321,32B,Airbus A321 (sharklets),Airbus
9,A21N,32Q,Airbus A321neo,Airbus


Not all information for these aircrafts are easily available on one file source or api.

The 4 remaining aircrafts with missing information will be searched manually through the internet and the information will be verified using a sample from the dataset whether the flights done by the aircraft was a passenger or cargo flight. 


In [70]:

B73P_flights = db.sql(
    """
    select *
    from vra_df
    where trim("Sigla ICAO Empresa Aérea") = 'AZU' and trim("Modelo Equipamento") = 'B73P'
    and trim("Código DI") = '0'
    """
).df()

A330_flights = db.sql(
    """
    select *
    from vra_df
    where trim("Sigla ICAO Empresa Aérea") = 'AZU' and trim("Modelo Equipamento") = 'A330'
    and trim("Código DI") = '0'
    """
).df()

AN22_flights = db.sql(
    """
    select *
    from vra_df
    where trim("Sigla ICAO Empresa Aérea") = 'AZU' and trim("Modelo Equipamento") = 'AN22'
    and trim("Código DI") = '0'
    """
).df()

C082_flights = db.sql(
    """
    select *
    from vra_df
    where trim("Sigla ICAO Empresa Aérea") = 'AZU' and trim("Modelo Equipamento") = 'C082'
    and trim("Código DI") = '0'
    """
).df()

In [10]:
vra_azu_aircrafts_csv.loc[vra_azu_aircrafts_csv['aircraft_icao'] == 'B73P', ['aircraft_iata', 'model_name', 'manufacturer']] = ['73P', 'Boeing 737-400 Freighter', 'Boeing']

In [11]:
vra_azu_aircrafts_csv.loc[vra_azu_aircrafts_csv['aircraft_icao'] == 'A330', ['aircraft_iata', 'model_name', 'manufacturer']] = ['330', 'Airbus A330', 'Airbus']

In [12]:
vra_azu_aircrafts_csv.loc[vra_azu_aircrafts_csv['aircraft_icao'] == 'AN22', ['aircraft_iata', 'model_name', 'manufacturer']] = ['A22', 'An-22 Antheus', 'Antonov']

In [13]:
vra_azu_aircrafts_csv.loc[vra_azu_aircrafts_csv['aircraft_icao'] == 'C082', ['aircraft_iata', 'model_name', 'manufacturer']] = ['Unidentified', 'Unidentified', 'Unidentified']

In [14]:
vra_azu_aircrafts_csv

,aircraft_icao,aircraft_iata,model_name,manufacturer
0,A20N,32N,Airbus A320neo,Airbus
1,A339,339,Airbus A330-900neo,Airbus
2,AT72,AT7,Aerospatiale/Alenia ATR 72,Aerospatiale
3,B734,734,Boeing 737-400,Boeing
4,A321,321,Airbus A321,Airbus
5,AT76,ATR,Aerospatiale/Alenia ATR 72-600,Aerospatiale
6,A320,32A,Airbus A320 (sharklets),Airbus
7,A359,359,Airbus A350-900,Airbus
8,A321,32B,Airbus A321 (sharklets),Airbus
9,A21N,32Q,Airbus A321neo,Airbus


Links for missing aircraft information:

`AN22` - https://skybrary.aero/aircraft/an22

`A330` - https://www.seatlink.com/tools/aircraft-equipment-codes/airbus/airbus-a330-330-a330/

`B73P` - https://www.seatlink.com/tools/aircraft-equipment-codes/boeing/boeing-737-400-freighter-73p-73f/

`C082` - No matching information on the internet.

The aircraft with icao code `C082` had scheduled flights that were intended to carry passengers. I could not find matching information from the internet regarding this aircraft icao code of `C082`.

In [15]:
vra_azu_aircrafts_csv.to_csv(r"C:\Users\USER\Desktop\anac_reg_flights\data_dictionary\vra_azu_aircrafts.csv",index=False)